In [62]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler
import random
import tensorflow.keras.utils as utils
import pydot
from tensorflow.keras import regularizers

In [63]:
dataset_dir = "Datasets/houseprices_ready.csv"

In [64]:
df = pd.read_csv(dataset_dir, index_col=0)
df = df.fillna(df.mean())
df.head(10)

,AboveMedianPrice,LotArea,OverallQual,OverallCond,TotalBsmtSF,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,Fireplaces,GarageArea
0,1,8450,7,5,856,2,1,3,8,0,548
1,1,9600,6,8,1262,2,0,3,6,1,460
2,1,11250,7,5,920,2,1,3,6,1,608
3,0,9550,7,5,756,1,0,3,7,1,642
4,1,14260,8,5,1145,2,1,4,9,1,836
5,0,14115,5,5,796,1,1,1,5,0,480
6,1,10084,8,5,1686,2,0,3,7,1,636
7,1,10382,7,6,1107,2,1,3,7,2,484
8,0,6120,7,5,952,2,0,2,8,2,468
9,0,7420,5,6,991,1,0,2,5,2,205


In [65]:
df.isnull().values.any()

False

In [68]:
df = df.sample(frac=1, random_state=41)

In [69]:
col_num = len(df.columns)
row_num = len(df.index)
print("Number of columns is: {} and number of rows is: {}".format(col_num, row_num))

Number of columns is: 11 and number of rows is: 1460


In [71]:
trainx = df.iloc[:int(0.8*row_num), 1:(col_num)]
trainy = df.iloc[:int(0.8*row_num), 0]

testx = df.iloc[int(0.8*row_num):, 1:(col_num)]
testy = df.iloc[int(0.8*row_num):, 0]

In [72]:
trainy

253     0
892     0
361     0
802     1
412     1
       ..
794     1
1296    0
684     1
1085    0
1455    1
Name: AboveMedianPrice, Length: 1168, dtype: int64

In [73]:
scaler = StandardScaler()
trainx_scaled = pd.DataFrame(scaler.fit_transform(trainx), columns = trainx.columns, index = trainx.index)
textx_scaled = pd.DataFrame(scaler.transform(testx), columns = testx.columns, index = testx.index)

In [74]:
trainx_scaled

,LotArea,OverallQual,OverallCond,TotalBsmtSF,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,Fireplaces,GarageArea
253,-0.113566,-0.091651,1.312542,0.548105,0.780426,-0.782453,0.161791,0.285697,-0.951804,-0.762229
892,-0.228165,-0.091651,2.226159,-0.000201,-1.035385,-0.782453,0.161791,-0.321019,-0.951804,-0.970466
361,-0.138788,-0.819873,-0.514692,-0.397328,-1.035385,-0.782453,0.161791,0.892413,-0.951804,-1.081526
802,-0.254489,0.636570,-0.514692,-0.747070,0.780426,1.208626,0.161791,0.285697,0.613981,-0.294852
412,-0.719250,0.636570,-0.514692,0.927181,0.780426,1.208626,-1.065298,0.285697,0.613981,0.047582
...,...,...,...,...,...,...,...,...,...,...
794,0.067882,0.636570,-0.514692,-0.783173,0.780426,1.208626,0.161791,0.285697,0.613981,-0.299480
1296,-0.193149,-0.819873,0.398925,0.137440,0.780426,-0.782453,0.161791,-0.321019,-0.951804,0.237310
684,0.794898,0.636570,-0.514692,0.306670,0.780426,1.208626,1.388879,0.285697,-0.951804,0.056837
1085,-0.147971,-0.091651,0.398925,-0.277739,-1.035385,-0.782453,-1.065298,-0.927736,-0.951804,0.417782


In [75]:
import random
def create_rand_list(max_val, count):
    randomlist = random.sample(range(0, max_val + 1), count)
    return randomlist

In [77]:
def get_MICS_model(inp_size, drop_out, hidden_num = 4, hidden_size=32):
    inputs = keras.layers.Input(shape=(inp_size), name="input")
        
    h = keras.layers.Dense(hidden_size, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3))(inputs)
    h = keras.layers.Dropout(drop_out)(h)
    for hidden in range(hidden_num):
        h = keras.layers.Dense(hidden_size, activation="relu", kernel_regularizer=regularizers.l1_l2(l1=1e-4, l2=1e-3))(h)
        h = keras.layers.Dropout(drop_out)(h) 

    outputs = keras.layers.Dense(1, activation="sigmoid")(h)    
    return keras.Model(inputs=[inputs], outputs = outputs)

In [78]:
trainx_scaled

,LotArea,OverallQual,OverallCond,TotalBsmtSF,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,Fireplaces,GarageArea
253,-0.113566,-0.091651,1.312542,0.548105,0.780426,-0.782453,0.161791,0.285697,-0.951804,-0.762229
892,-0.228165,-0.091651,2.226159,-0.000201,-1.035385,-0.782453,0.161791,-0.321019,-0.951804,-0.970466
361,-0.138788,-0.819873,-0.514692,-0.397328,-1.035385,-0.782453,0.161791,0.892413,-0.951804,-1.081526
802,-0.254489,0.636570,-0.514692,-0.747070,0.780426,1.208626,0.161791,0.285697,0.613981,-0.294852
412,-0.719250,0.636570,-0.514692,0.927181,0.780426,1.208626,-1.065298,0.285697,0.613981,0.047582
...,...,...,...,...,...,...,...,...,...,...
794,0.067882,0.636570,-0.514692,-0.783173,0.780426,1.208626,0.161791,0.285697,0.613981,-0.299480
1296,-0.193149,-0.819873,0.398925,0.137440,0.780426,-0.782453,0.161791,-0.321019,-0.951804,0.237310
684,0.794898,0.636570,-0.514692,0.306670,0.780426,1.208626,1.388879,0.285697,-0.951804,0.056837
1085,-0.147971,-0.091651,0.398925,-0.277739,-1.035385,-0.782453,-1.065298,-0.927736,-0.951804,0.417782


In [79]:
inp_size = len(test_x_rand.columns)

In [80]:
min_losses = {12:[], 14:[], 16:[], 18:[], 20:[], 22:[], 24:[]}

In [83]:
inp_size = len(test_x_rand.columns)
MICS_model = get_MICS_model(inp_size, drop_out = 0.25)
callback = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50), 
        keras.callbacks.ReduceLROnPlateau("val_loss", factor = 0.8, patience=30,
                                         verbose = 2, mode = "auto", 
                                          min_lr = 1e-6)]
MICS_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01), loss=keras.losses.BinaryCrossentropy(), metrics=["accuracy"])
history = MICS_model.fit(x = [trainx_scaled], y = trainy.values,  
                         validation_data = ([textx_scaled], testy.values),
                         epochs=300, batch_size = 32, callbacks=callback)
training_val_accuracy = history.history["val_accuracy"]
best_row_index = np.argmax(training_val_accuracy)
best_val_accuracy = training_val_accuracy[best_row_index]
best_val_accuracy
#min_losses3[c].append(best_val_loss)
#i = i + 1

Epoch 1/300
37/37 [==============================] - 0s 4ms/step - loss: 0.6227 - accuracy: 0.8151 - val_loss: 0.4150 - val_accuracy: 0.9007
Epoch 2/300
37/37 [==============================] - 0s 1ms/step - loss: 0.4351 - accuracy: 0.8887 - val_loss: 0.3784 - val_accuracy: 0.9075
Epoch 3/300
37/37 [==============================] - 0s 1ms/step - loss: 0.3812 - accuracy: 0.8990 - val_loss: 0.3582 - val_accuracy: 0.9144
Epoch 4/300
37/37 [==============================] - 0s 1ms/step - loss: 0.3664 - accuracy: 0.9058 - val_loss: 0.3512 - val_accuracy: 0.8938
Epoch 5/300
37/37 [==============================] - 0s 1ms/step - loss: 0.3490 - accuracy: 0.9101 - val_loss: 0.3576 - val_accuracy: 0.8836
Epoch 6/300
37/37 [==============================] - 0s 1ms/step - loss: 0.3407 - accuracy: 0.9024 - val_loss: 0.3242 - val_accuracy: 0.9007
Epoch 7/300
37/37 [==============================] - 0s 1ms/step - loss: 0.3242 - accuracy: 0.9144 - val_loss: 0.3285 - val_accuracy: 0.8904
Epoch 8/300
3

0.914383590221405